### Import Libraries

In [ ]:
"""
Two different prices are being tested at the same time for the same buyer. The performance of each is measured by 4 metrics: the number of sales, revenue, payout, and earnings.

Part 1: Based on the data, discover if any of the two price options shows a significantly better performance than the other one. Write a script that will determine which test category is the "winner" using any programming language that you would like.

Part 2: Write a brief explanation of how you would design a system that would optimize 500 tests like this every day. 
"""

In [122]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import seaborn as sns
from math import ceil
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

### Read the data & EDA

In [3]:
df = pd.read_excel("Data_Engineer_Data.xlsx")

In [121]:
df.head()

,split_test_option,accept_status,revenue,payout,erns
0,[7951],0,0.0,0.0,0.0
1,[7951],0,0.0,0.0,0.0
2,[7951],0,0.0,0.0,0.0
3,[7952],0,0.0,0.0,0.0
4,[7951],0,0.0,0.0,0.0


In [7]:
df.split_test_option.value_counts()

split_test_option
[7951]    8944
[7952]    2564
Name: count, dtype: int64

In [8]:
df.accept_status.value_counts()

accept_status
0    11438
1       70
Name: count, dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11508 entries, 0 to 11507
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   split_test_option  11508 non-null  object 
 1   accept_status      11508 non-null  int64  
 2   revenue            11508 non-null  float64
 3   payout             11508 non-null  float64
 4   erns               11508 non-null  float64
dtypes: float64(3), int64(1), object(1)
memory usage: 449.7+ KB


In [128]:
def percentCount(df):
    counts = df.accept_status.value_counts()
    percent = df.accept_status.value_counts(normalize=True)
    percent100 = df.accept_status.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
    return pd.DataFrame({'counts': counts, 'per': percent, 'per100': percent100})

In [129]:
percentCount(df_7951)

,counts,per,per100
accept_status,,,
0,8893,0.994298,99.4%
1,51,0.005702,0.6%


In [130]:
percentCount(df_7952)

,counts,per,per100
accept_status,,,
0,2545,0.99259,99.3%
1,19,0.00741,0.7%


In [127]:
df.groupby('split_test_option').describe().transpose()

split_test_option         [7951]       [7952]
accept_status count  8944.000000  2564.000000
              mean      0.005702     0.007410
              std       0.075301     0.085780
              min       0.000000     0.000000
              25%       0.000000     0.000000
              50%       0.000000     0.000000
              75%       0.000000     0.000000
              max       1.000000     1.000000
revenue       count  8944.000000  2564.000000
              mean      0.100846     0.060920
              std       1.847549     1.100745
              min       0.000000     0.000000
              25%       0.000000     0.000000
              50%       0.000000     0.000000
              75%       0.000000     0.000000
              max      78.050000    40.070000
payout        count  8944.000000  2564.000000
              mean      0.077616     0.045593
              std       1.582549     0.931970
              min       0.000000     0.000000
              25%       0.000000     0.000000
              50%       0.000000     0.000000
              75%       0.000000     0.000000
              max      74.100000    40.000000
erns          count  8944.000000  2564.000000
              mean      0.023230     0.015328
              std       0.799460     0.581777
              min       0.000000     0.000000
              25%       0.000000     0.000000
              50%       0.000000     0.000000
              75%       0.000000     0.000000
              max      40.070000    27.230000

### A/B Test

In [115]:
def evalTests(df, metric):
    # Define functions for standard deviation and standard error
    std_dev = lambda x : np.std(x, ddof = 0) 
    std_error = lambda x : stats.sem(x, ddof = 0)
    conversion_rate = df.groupby('split_test_option')[metric].agg([np.mean, std_dev, std_error])
    conversion_rate.columns = ['conversion_rate', 'std_deviation', 'std_error']
    print(conversion_rate)
    

    test_1_results = df[df['split_test_option'] == '[7951]'][metric]
    test_2_results = df[df['split_test_option'] == '[7952]'][metric]

    num_test1 = test_1_results.count()
    num_test2 = test_2_results.count()
    successes = [test_1_results.sum(), test_2_results.sum()]
    nobs = [num_test1, num_test2]

    z_stat, pval = proportions_ztest(successes, nobs = nobs)
    (lower_test1, lower_test2), (upper_test1, upper_test2) = proportion_confint(successes, nobs=nobs, alpha=0.05)

    print(f'Z Statistic - {z_stat:.2f}')
    print(f'P-Value - {pval:.3f}')
    print(f'CI 95% for test1 group - [{lower_test1:.3f}, {upper_test1:.3f}]')
    print(f'CI 95% for test2 group - [{lower_test2:.3f}, {upper_test2:.3f}]')

In [132]:
print("Test Results for each metrics:", "\n")
for metric in ['accept_status', 'revenue', 'payout', 'erns']:
    print(metric)
    evalTests(df, metric)
    print("\n")

Test Results for each metrics: 

accept_status
                   conversion_rate  std_deviation  std_error
split_test_option                                           
[7951]                    0.005702       0.075297   0.000796
[7952]                    0.007410       0.085764   0.001694
Z Statistic - -0.98
P-Value - 0.327
CI 95% for test1 group - [0.004, 0.007]
CI 95% for test2 group - [0.004, 0.011]


revenue
                   conversion_rate  std_deviation  std_error
split_test_option                                           
[7951]                    0.100846       1.847446   0.019535
[7952]                    0.060920       1.100530   0.021734
Z Statistic - 6.17
P-Value - 0.000
CI 95% for test1 group - [0.095, 0.107]
CI 95% for test2 group - [0.052, 0.070]


payout
                   conversion_rate  std_deviation  std_error
split_test_option                                           
[7951]                    0.077616       1.582461   0.016733
[7952]                    0.0455

### Conclusion
Although we don't see a significance difference(alpha being 0.05) in the number of sales between two price options, when we look at the earnings - which we may consider as the main metric for this study, we see a significant lift using the price option $[7951]$ comparing the $[7952]$. So based on this test, we can infer that $[7951]$ will be a more profitable option.   